In [1]:
import cPickle
import numpy as np
import os
from PIL import Image
import argparse
import chainer
from chainer import cuda, Variable, FunctionSet, optimizers, serializers ### Add 'serializers'
import chainer.functions  as F
import chainer.links as L
import pdb
import argparse
from Original_VGGNet import Original_VGGNet
import glob
from sklearn.cross_validation import train_test_split
import cPickle as pickle
import cv2
#parser = argparse.ArgumentParser()
#parser.add_argument('--gpu',default=0,help='GPU ID (default 0)')
#parser.add_argument('--model',default=0,help='use pre-trained model or not')
#parser.add_argument('--load',default=1,help='LOAD or not (default 1)')
#args = parser.parse_args()


classes =  ["aragaki_face","hoshino_face","narita_face","other_face","fujii_face","mano_face","ohtani_face","yamaga_face","ishida_face"]
#load=int(args.load)

class VGGNet(chainer.Chain):

    """
    VGGNet
    - It takes (224, 224, 3) sized image as imput
    """

    def __init__(self):
        super(VGGNet, self).__init__(
            conv1_1=L.Convolution2D(3, 64, 3, stride=1, pad=1),
            conv1_2=L.Convolution2D(64, 64, 3, stride=1, pad=1),

            conv2_1=L.Convolution2D(64, 128, 3, stride=1, pad=1),
            conv2_2=L.Convolution2D(128, 128, 3, stride=1, pad=1),

            conv3_1=L.Convolution2D(128, 256, 3, stride=1, pad=1),
            conv3_2=L.Convolution2D(256, 256, 3, stride=1, pad=1),
            conv3_3=L.Convolution2D(256, 256, 3, stride=1, pad=1),

            conv4_1=L.Convolution2D(256, 512, 3, stride=1, pad=1),
            conv4_2=L.Convolution2D(512, 512, 3, stride=1, pad=1),
            conv4_3=L.Convolution2D(512, 512, 3, stride=1, pad=1),

            conv5_1=L.Convolution2D(512, 512, 3, stride=1, pad=1),
            conv5_2=L.Convolution2D(512, 512, 3, stride=1, pad=1),
            conv5_3=L.Convolution2D(512, 512, 3, stride=1, pad=1),

            fc6=L.Linear(7*7*512, 4096),
            fc7=L.Linear(4096, 4096),
            fc8=L.Linear(4096,9)
            # the output dim '10' is set up to CIFAR-10. Please change this.
        )

    def __call__(self, x, t,train):
        h = F.relu(self.conv1_1(x))
        h = F.relu(self.conv1_2(h))
        h = F.max_pooling_2d(h, 2, stride=2)

        h = F.relu(self.conv2_1(h))
        h = F.relu(self.conv2_2(h))
        h = F.max_pooling_2d(h, 2, stride=2)

        h = F.relu(self.conv3_1(h))
        h = F.relu(self.conv3_2(h))
        h = F.relu(self.conv3_3(h))
        h = F.max_pooling_2d(h, 2, stride=2)

        h = F.relu(self.conv4_1(h))
        h = F.relu(self.conv4_2(h))
        h = F.relu(self.conv4_3(h))
        h = F.max_pooling_2d(h, 2, stride=2)

        h = F.relu(self.conv5_1(h))
        h = F.relu(self.conv5_2(h))
        h = F.relu(self.conv5_3(h))
        h = F.max_pooling_2d(h, 2, stride=2)

        h = F.dropout(F.relu(self.fc6(h)), train=train, ratio=0.5)
        h = F.dropout(F.relu(self.fc7(h)), train=train, ratio=0.5)

        h = self.fc8(h)

        self.loss = F.softmax_cross_entropy(h, t)
        self.acc = F.accuracy(h, t)
        self.pred = F.softmax(h)
        return self.loss,self.acc,self.pred

def unpickle(file):
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

def conv_setup(ORIGINAL_VGG,VGG):
    VGG.conv1_1 = ORIGINAL_VGG.conv1_1
    VGG.conv1_2 = ORIGINAL_VGG.conv1_2
    VGG.conv2_1 = ORIGINAL_VGG.conv2_1
    VGG.conv2_2 = ORIGINAL_VGG.conv2_2
    VGG.conv3_1 = ORIGINAL_VGG.conv3_1
    VGG.conv3_2 = ORIGINAL_VGG.conv3_2
    VGG.conv3_3 = ORIGINAL_VGG.conv3_3
    VGG.conv4_1 = ORIGINAL_VGG.conv4_1
    VGG.conv4_2 = ORIGINAL_VGG.conv4_2
    VGG.conv4_3 = ORIGINAL_VGG.conv4_3
    VGG.conv5_1 = ORIGINAL_VGG.conv5_1
    VGG.conv5_2 = ORIGINAL_VGG.conv5_2
    VGG.conv5_3 = ORIGINAL_VGG.conv5_3
    VGG.fc6=ORIGINAL_VGG.fc6
    VGG.fc7=ORIGINAL_VGG.fc7
    return VGG


def preprocess(x_train, x_test):
    print(x_train.shape)
    mean = np.mean(x_train, axis=(0, 2, 3), keepdims=True)
    np.save("mean_face.npy",mean)
    return x_train-mean, x_test-mean

def augment(x,size):#4D tensor
    #flip = np.random.randint(2,size = len(x))*2-1
    theta = np.random.uniform(-0.1*np.pi,0.1*np.pi,len(x))
    scale = np.random.uniform(0.9,1.2,len(x)).reshape(-1,1,1)
    shift = np.random.uniform(-5,5,len(x)*2).reshape(-1,2)
    xs = np.arange(size**2)%size
    ys = np.arange(size**2)/size
    coords = np.c_[xs,ys].transpose()-size/2.#変換前の座標
    R = scale*(np.c_[np.cos(theta),-np.sin(theta),np.sin(theta),np.cos(theta)].reshape(len(x),2,2))
    img = np.array([X[:,np.clip(np.dot(r,coords)[1]+size/2.+s[1],0,size-1).astype('int32'),np.clip(np.dot(r,coords)[0]+size/2.+s[0],0,size-1).astype('int32')].reshape(3,size,size) for s,r,X in zip(shift,R,x)])
    return img

def load_faces():
    data_dir = "data/faces_nparray/"
    classes =  ["aragaki_face","hoshino_face","narita_face","other_face","fujii_face","mano_face","ohtani_face","yamaga_face","ishida_face"]
    X=[]
    y=[]
    
    for i, cls in enumerate(classes):
        X_=np.load(data_dir+cls+".npy").astype(np.uint8)
        kkk=np.array([cv2.resize(ll,((224,224))).transpose(2,0,1)/255. for ll in X_])
        X.extend(kkk)
        y.extend(np.ones(len(kkk),dtype=np.int32)*i)
    X = np.array(X,dtype=np.float32)
    y = np.array(y,dtype=np.int32)

    return X,y

In [ ]:
def valid_train():
    with open('vgg.pkl', 'rb') as i:
        orig_vgg = pickle.load(i)
    vgg=VGGNet()
    #if int(args.model) == 1:
    #orig_vgg = Original_VGGNet()
    #serializers.load_hdf5('VGG.model', orig_vgg)
    vgg = conv_setup(orig_vgg,vgg)
    print 'loading data now.'
    
    #if not load :
    X,y = load_faces()
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    
    print 'loading data done'
    x_train,x_test = preprocess(x_train,x_test)
    batchsize = 5
    N = len(x_train)
    N_test = len(x_test)
    n_epoch = 50
    #gpu = int(args.gpu)
    optimizer = optimizers.MomentumSGD(lr=0.002,momentum=0.9)
    
    optimizer.setup(vgg)
    optimizer.add_hook(chainer.optimizer.WeightDecay(0.001))
    vgg.to_gpu()
    
    for epoch in xrange(1, n_epoch+1):
        
        if epoch in [35,45]:
            optimizer.lr*=0.20
        print 'epoch', epoch
        # training
        perm = np.random.permutation(N)
        sum_loss = 0
        for i in xrange(0, N, batchsize):
            x_batch = cuda.to_gpu(augment(x_train[perm[i:i+batchsize]],224))
            y_batch = cuda.to_gpu(y_train[perm[i:i+batchsize]])
            optimizer.zero_grads()
            loss,_,_ = vgg(x_batch, y_batch, train=True)
            loss.backward()
            optimizer.update()
            sum_loss     += float(cuda.to_cpu(loss.data)) * len(x_batch)

        print 'train mean loss={}'.format(sum_loss / N)
        
        # evaluation
        sum_accuracy = 0
        
        pred_y =[]
        for i in xrange(0, N_test, batchsize):
            x_batch = cuda.to_gpu(x_test[i:i+batchsize])
            y_batch = cuda.to_gpu(y_test[i:i+batchsize])

            _,acc,pred = vgg(x_batch,y_batch,train=False)
            pred_y.extend(np.argmax(cuda.to_cpu(pred.data),axis=1))
            sum_accuracy += float(cuda.to_cpu(acc.data)) * len(x_batch)

        print 'test mean accuracy={}'.format(sum_accuracy / N_test)
        for i in range(len(classes)):
            accuracy = np.sum((np.array(pred_y)==y_test)*(y_test==i))*1./np.sum(y_test==i)
            print '    {} accuracy={}'.format(classes[i],accuracy)

In [ ]:
def train():
    with open('vgg.pkl', 'rb') as i:
        orig_vgg = pickle.load(i)
    vgg=VGGNet()
    #if int(args.model) == 1:
    #orig_vgg = Original_VGGNet()
    #serializers.load_hdf5('VGG.model', orig_vgg)
    vgg = conv_setup(orig_vgg,vgg)
    print 'loading data now.'
    
    #if not load :
    X,y_train = load_faces()
    
    print 'loading data done'
    x_train,_= preprocess(X,[])
    batchsize = 5
    N = len(x_train)
    n_epoch = 50
    #gpu = int(args.gpu)
    optimizer = optimizers.MomentumSGD(lr=0.002,momentum=0.9)
    
    optimizer.setup(vgg)
    optimizer.add_hook(chainer.optimizer.WeightDecay(0.001))
    vgg.to_gpu()
    
    for epoch in xrange(1, n_epoch+1):
        
        if epoch in [35,45]:
            optimizer.lr*=0.20
            
        print 'epoch', epoch
        # training
        perm = np.random.permutation(N)
        sum_loss = 0
        for i in xrange(0, N, batchsize):
            x_batch = cuda.to_gpu(augment(x_train[perm[i:i+batchsize]],224))
            y_batch = cuda.to_gpu(y_train[perm[i:i+batchsize]])
            optimizer.zero_grads()
            loss,_,_ = vgg(x_batch, y_batch, train=True)
            loss.backward()
            optimizer.update()
            sum_loss     += float(cuda.to_cpu(loss.data)) * len(x_batch)

        print 'train mean loss={}'.format(sum_loss / N)
        
    serializers.save_hdf5('VGGface_{}.model'.format(str(sum_loss/N).replace('.','')), vgg)

In [2]:
def load_testFaces():
    dirs = ["aragaki","hoshino","narita","furuta","fujii","mano","ohtani","yamaga","ishida"]
    #X = np.empty((0, 3, 224, 224))
    X = []
    y = []
    for i,dir in enumerate(dirs):
        data_dir="./data/kaokai/"+dir+"/"
        for f in sorted(glob.glob(data_dir+"/*.jpg")):
            x = np.array(Image.open(f).resize((224, 224))) 
            x = np.transpose(x, (2, 0, 1)) / 255. 
            X.append(x)
            y.append(i)
    y=np.array(y,dtype=np.int32)
    X=np.array(X,dtype=np.float32)
    return X,y

def test():
    model=VGGNet()
    model.to_gpu()
    serializers.load_hdf5("VGGface_00010659227118.model",model)
    #if int(args.model) == 1:
    #orig_vgg = Original_VGGNet()
    #serializers.load_hdf5('VGG.model', orig_vgg)
    #vgg = conv_setup(orig_vgg,vgg)
    print 'loading data now.'
    #if not load :
    X,y = load_testFaces()
    print "data loaded"
    N_test=len(y)
    mean=np.load("mean_face.npy")
    X-=mean
    batch_num=10
    pred=[]
    for i in range(0,N_test,batch_num):
        x=Variable(cuda.to_gpu(X[i:i+batch_num]))
        t=Variable(cuda.to_gpu(y[i:i+batch_num]))
        _,_,pre=model(x,t,train=False)
        pred.extend(cuda.to_cpu(pre.data))
    classes=["aragaki","hoshino","narita","furuta","fujii","mano","ohtani","yamaga","ishida"]
    prediction=np.argmax(pred,axis=1)
    print(prediction)
    accuracy = np.sum(np.eye(10)[prediction] * np.eye(10)[y]) / N_test
    print(accuracy)
    indices=[]
    for i in range(len(classes)):
        print(classes[i]+"accuracy : "+str(float(np.sum((y==i)*(prediction==i)))/np.sum(y==i)*100)+"%")
        indices.append(list(set(np.where(y==i)[0]).difference(set(np.where((y==i)*(prediction==i))[0]))))
    np.save("indices.npy",np.array(indices))

In [3]:
if __name__ =='__main__':
    test()

/home/mil/tanaka/.pyenv/versions/anaconda2-4.0.0/lib/python2.7/site-packages/chainer/cuda.py:90: UserWarning: cuDNN is not enabled.
Please reinstall chainer after you install cudnn
(see https://github.com/pfnet/chainer#installation).
  'cuDNN is not enabled.\n'


loading data now.
data loaded
[0 0 0 ..., 0 5 5]
0.786353944563
aragakiaccuracy : 85.4545454545%
hoshinoaccuracy : 98.8235294118%
naritaaccuracy : 96.9696969697%
furutaaccuracy : 93.2835820896%
fujiiaccuracy : 100.0%
manoaccuracy : 73.3333333333%
ohtaniaccuracy : 96.6666666667%
yamagaaccuracy : 89.2857142857%
ishidaaccuracy : 36.3449691992%


In [ ]:
X,y_train = load_faces()
mean = np.mean(X, axis=(0, 2, 3), keepdims=True)
np.save("mean.npy",mean)

In [4]:
indices=np.load("indices.npy")
print(indices)

[ [1037, 1038, 1044, 686, 1046, 1047, 539, 858, 945, 861, 587, 605, 612, 617, 618, 619, 620, 621, 622, 623, 624, 625, 628, 629, 123, 124, 639, 640, 642, 643, 649, 650, 651, 993, 666, 1045, 673, 674, 174, 687, 689, 194, 220, 741, 752, 758, 759, 760, 251, 252, 258, 259, 261, 262, 263, 264, 265, 783, 795, 817, 315, 316, 833, 834, 835, 837, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 364, 365, 366, 367, 368, 369, 370, 371, 372, 377, 381, 385, 386, 389, 390, 391, 392, 394, 395, 397, 410, 923, 412, 925, 416, 419, 420, 422, 935, 424, 924, 938, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 441, 442, 444, 842, 446, 447, 961, 931, 932, 848, 934, 954, 936, 1021]
 [1152, 1195, 1277, 1238] [1445]
 [1571, 1572, 1573, 1604, 1589, 1561, 1596, 1597, 1598] []
 [1632, 1645, 1646, 1647, 1648, 1649, 1629, 1631]
 [1821, 1657, 1658, 1819, 1820, 1818] [1841, 184